# Feature Engineering - Skater + Goalie Data

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns

## Read in data from hockey reference

In [2]:
# Skater data
skaters_2021 = pd.read_csv("../data/skaters_2021.csv")
skaters_2022 = pd.read_csv("../data/skaters_2022.csv")

# Goalie data
goalies_2021 = pd.read_csv("../data/goalies_2021.csv")
goalies_2022 = pd.read_csv("../data/goalies_2022.csv")

In [54]:
skaters_2021.isnull().sum(axis = 0)
#skaters_2022.isnull().sum(axis = 0)
#goalies_2021.isnull().sum(axis = 0)
#goalies_2022.isnull().sum(axis = 0)

player_id               0
player_name             0
age                     0
season                  0
game_num                0
date                    0
team                    0
opponent                0
home_away_status        0
result                  0
G                       0
A                       0
P                       0
rating                  0
PIM                     0
EVG                     0
PPG                     0
SHG                     0
GWG                     0
EVA                     0
PPA                     0
SHA                     0
S                       0
S_perc               7330
shifts                  0
TOI                     0
HIT                     0
BLK                     0
FOW                     0
FOL                     0
FOW_perc            19451
dtype: int64

## Function for creating skaters final data
Should create all the cumulative stats for the skaters.

### Setting up a place to store cumulative computed features
Ideally, this will basically be the data frame containing the rows/observations to train a model on. We can include all of the same information like player ID, game number, team, etc. But for the features like shots or TOI, we will compute the cumulative sums/averages up to the start of each game and store them here. This way, we can keep separate the raw game by game data from the cumulative data.

In [80]:
skaters_2021.head(15)
skaters_2021.sort_values(['player_id', 'game_num']).head(15)

,player_id,player_name,age,season,game_num,date,team,opponent,home_away_status,result,...,SHA,S,S_perc,shifts,TOI,HIT,BLK,FOW,FOL,FOW_perc
0,/a/abramvi01,Vitaly Abramov,22,2021,1,2021-05-05,OTT,MTL,1,W,...,0,0,NaN,12,9.633333,0,0,0,0,NaN
1,/a/abramvi01,Vitaly Abramov,23,2021,2,2021-05-08,OTT,WPG,0,W,...,0,0,NaN,12,9.800000,0,0,0,0,NaN
2,/a/acciano01,Noel Acciari,29,2021,1,2021-01-17,FLA,CHI,1,W,...,0,1,0.0,21,14.633333,2,1,5,10,33.3
3,/a/acciano01,Noel Acciari,29,2021,2,2021-01-19,FLA,CHI,1,W,...,0,0,NaN,23,15.266667,3,3,5,10,33.3
4,/a/acciano01,Noel Acciari,29,2021,3,2021-01-26,FLA,CBJ,0,W,...,0,0,NaN,25,17.200000,1,3,12,7,63.2
5,/a/acciano01,Noel Acciari,29,2021,4,2021-01-28,FLA,CBJ,0,L-SO,...,0,1,0.0,23,14.966667,5,0,5,7,41.7
6,/a/acciano01,Noel Acciari,29,2021,5,2021-01-30,FLA,DET,0,W,...,0,0,NaN,21,13.966667,4,1,5,2,71.4
7,/a/acciano01,Noel Acciari,29,2021,6,2021-01-31,FLA,DET,0,W,...,0,3,0.0,23,18.266667,2,1,6,6,50.0
8,/a/acciano01,Noel Acciari,29,2021,7,2021-02-04,FLA,NSH,1,L-OT,...,0,1,0.0,18,15.033333,4,2,0,4,0.0
9,/a/acciano01,Noel Acciari,29,2021,8,2021-02-05,FLA,NSH,1,W,...,0,1,0.0,19,11.350000,3,3,2,3,40.0


In [115]:
test = skaters_2021[skaters_final_2022['player_name'] == 'Parker Kelly'].apply(lambda x: skaters_2021[(skaters_2021.player_id == x.player_id) & (skaters_2021.game_num < x.game_num)][['G', 'A', 'P', 'rating',
       'PIM', 'EVG', 'PPG', 'SHG', 'GWG', 'EVA', 'PPA', 'SHA', 'S', 'shifts', 'TOI', 'HIT', 'BLK', 'FOW', 'FOL']].agg(np.sum), axis=1)
#test_num_games = skaters_2021.head(15).apply(lambda x: skaters_2021[(skaters_2021.player_id == x.player_id) & (skaters_2021.game_num < x.game_num)]['game_num'].agg('count'), axis=1)
cols = [col for col in test.columns if col != 'TOI']
test2 = test[cols].div(test['TOI'], axis = 0)
test2 = 60 * test2
test2.columns = [col + '_60' for col in cols]
test2['total_TOI'] = test['TOI']
#test2['n_completed_games'] = test_num_games
#test2['avg_TOI'] = test2['total_TOI'] / test2['n_completed_games']
display(test2)

/var/folders/yk/qhxv13m50bl7fyrpnntydjgh0000gn/T/ipykernel_39023/2475958955.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = skaters_2021[skaters_final_2022['player_name'] == 'Parker Kelly'].apply(lambda x: skaters_2021[(skaters_2021.player_id == x.player_id) & (skaters_2021.game_num < x.game_num)][['G', 'A', 'P', 'rating',


,G_60,A_60,P_60,rating_60,PIM_60,EVG_60,PPG_60,SHG_60,GWG_60,EVA_60,PPA_60,SHA_60,S_60,shifts_60,HIT_60,BLK_60,FOW_60,FOL_60,total_TOI
22112,0.101557,0.914015,1.015572,-0.203114,1.421801,0.101557,0.0,0.0,0.000000,0.914015,0.000000,0.0,5.179418,78.909953,3.148274,5.890318,0.000000,0.000000,590.800000
22113,0.097297,0.875676,0.972973,-0.389189,1.362162,0.097297,0.0,0.0,0.000000,0.875676,0.000000,0.0,5.156757,78.616216,3.113514,5.740541,0.000000,0.000000,616.666667
22114,0.187975,0.845887,1.033862,-0.375950,1.315824,0.187975,0.0,0.0,0.000000,0.845887,0.000000,0.0,5.169308,78.573480,3.101585,5.545258,0.000000,0.000000,638.383333
22115,0.181988,0.818947,1.000935,-0.363976,1.273918,0.181988,0.0,0.0,0.000000,0.818947,0.000000,0.0,5.277658,78.618912,3.093800,5.641635,0.000000,0.000000,659.383333
22116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
22117,0.000000,0.000000,0.000000,2.832415,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,5.664831,62.313139,0.000000,0.000000,8.497246,25.491739,21.183333
22118,0.000000,0.000000,0.000000,0.000000,3.241783,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,8.104457,68.077443,1.620891,0.000000,12.967132,16.208915,37.016667
22119,0.000000,0.000000,0.000000,0.000000,2.177858,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,10.889292,64.246824,1.088929,0.000000,11.978221,15.245009,55.100000
22120,0.000000,0.000000,0.000000,-0.825499,1.650997,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,10.731484,61.912405,0.825499,2.476496,11.556982,13.207980,72.683333
22121,0.672520,0.000000,0.672520,-0.672520,1.345040,0.672520,0.0,0.0,0.672520,0.000000,0.000000,0.0,10.760321,63.216888,0.672520,2.690080,10.760321,14.122922,89.216667


In [117]:
def create_cumulative_statistics(indiv_games, player_type):
    # Make sure data is in correct order
    indiv_games = indiv_games.sort_values(['player_id', 'game_num'])
    
    if player_type == "skater":
        # Create new data frames containing only certain columns from the old data frames
        cumulative_games = indiv_games.loc[:, ["player_id", "player_name", "age", "season", "game_num", "date", 
                                                        "team", "opponent", "home_away_status", "result", "G"]]
        
        # Assign the number of completed games as 1 less than game num
        cumulative_games['n_completed_games'] = cumulative_games['game_num'] - 1

        # For each numerical statistic, compute the sum of that statistic for all games completed by the player prior to the game that is about to start
        prior_stats_totals = indiv_games.apply(lambda x: indiv_games[(indiv_games.player_id == x.player_id) & (indiv_games.game_num < x.game_num)][['G', 'A', 'P', 'rating', 
                                                                                                                                                       'PIM', 'EVG', 'PPG', 'SHG', 'GWG', 'EVA', 'PPA', 'SHA', 'S', 'shifts', 'TOI', 'HIT', 'BLK', 'FOW', 'FOL']].agg(np.sum), axis=1)
        
        # Select all columns except TOI. We will compute the stat/60 min for the other columns, but leave the total TOI column unchanged.
        cols = [col for col in prior_stats_totals.columns if col != 'TOI']

        # Divide to compute the per 60 minute statistics
        prior_stats_60 = prior_stats_totals[cols].div(prior_stats_totals['TOI'], axis = 0)
        prior_stats_60 = 60 * prior_stats_60
        prior_stats_60.columns = [col + '_60' for col in cols]

        # Add total TOI back in
        prior_stats_final = prior_stats_60.copy()
        prior_stats_final['total_TOI'] = prior_stats_totals['TOI']

        # Concatenate with cumulative games
        cumulative_games = pd.concat([cumulative_games, prior_stats_final], axis = 1)

        # Calculate average TOI column
        # This gives us a final data set
        cumulative_games['avg_TOI'] = cumulative_games['total_TOI'] / cumulative_games['n_completed_games']
        
        
    elif player_type == "goalie":
        # Create new data frames containing only certain columns from the old data frames
        cumulative_games = indiv_games.loc[:, ["player_id", "player_name", "age", "season", "game_num", "date", 
                                                    "team", "opponent", "home_away_status", "result", "decision"]]
    
        # Compute Save Percentage = sum(Saves) / sum(Shots Against)
        cumulative_games[["SA", "SV"]] = indiv_games.groupby("player_id")[["SA", "SV"]].cumsum()
        cumulative_games["SV_perc"] = cumulative_games.SV / cumulative_games.SA
        cumulative_games = cumulative_games.drop(columns = ["SV", "SA"])
        
        # Compute goals against average?
        
    else:
        # If wrong player type, print warning message 
        print("The player type is not known. Should be one of either 1) skater 2) goalie.")
        return
    
    return cumulative_games

In [118]:
# Use function to create final data frames for skaters and goalies
skaters_final_2021 = create_cumulative_statistics(skaters_2021, player_type = "skater")
skaters_final_2022 = create_cumulative_statistics(skaters_2022, player_type = "skater")
#goalies_final_2021 = create_cumulative_statistics(goalies_2021, player_type = "goalie")
#goalies_final_2022 = create_cumulative_statistics(goalies_2022, player_type = "goalie")

In [94]:
#goalies_final_2021.head()

## Issue: On some days, a goalie played bad and was pulled from the game
This matters because once the 2nd goalie is put in, there are now 2 goalies that have played in the exact same game. Therefore, when we left join the goalie information to the skater information, we will be duplicating a lot of rows in the skater data frame (becuase there will be 2 rows in the goalie data frame that have the same date and team information as 1 row in the skater data. 

Extending/elaborating on this, we do not know which goalie the skaters scored on during this game. This poses a problem since we want to include the strength of the opposing teams goalie in the training data. If we don't know which goalie was in the game for the times that a player was on the ice, there is some innacuracy here.

There are 116 cases of a goalie getting pulled in the 2021 season.

Here is an example.

In [45]:
#goalies_final_2021.groupby(["date", "team"])["player_id"].count()
pulled_goalies = goalies_final_2021.groupby(["date", "team"])["player_id"].nunique()
pulled_goalies[pulled_goalies >= 2]

date        team
2021-01-15  PIT     2
            STL     2
2021-01-16  SJS     2
2021-01-18  PHI     2
2021-01-19  BUF     2
                   ..
2021-05-06  EDM     2
            MTL     2
2021-05-08  NYR     2
2021-05-09  OTT     2
2021-05-10  NYI     2
Name: player_id, Length: 116, dtype: int64

In [46]:
goalies_2021.loc[(goalies_2021.team == "PIT") & (goalies_2021.date == "2021-01-15"), :]

,player_id,player_name,age,season,game_num,date,team,opponent,home_away_status,result,decision,GA,SA,SV,SV_perc,shutout,PIM,TOI
283,/d/desmica01,Casey DeSmith,29,2021,1,2021-01-15,PIT,PHI,0,L,NaN,1,13,12,0.923,0,0,47.75
752,/j/jarrytr01,Tristan Jarry,25,2021,2,2021-01-15,PIT,PHI,0,L,L,3,6,3,0.500,0,0,11.50


## Left join goalie to skater?